# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-2718eeacfe-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Tennis explorative search") 

Consider the following exploratory information need:

> investigate tennis and find the main BGPs related to this sport. Compare players per countries, awards of the players and tournament.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q1426`| Roger Federer       | node |
| `wd:Q11459`| Serena Williams       | node |





Also consider

```
wd:Q1426 ?p ?obj .
```

is the BGP to retrieve all **properties of Roger Federer**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for tennis player

2. Identify the BGP for tennis tournament

3. Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

4. Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

5. Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

6. Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

7. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

8. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

## Task 1
Identify the BGP for tennis player

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1426 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1284'), ('name', 'Munzinger person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('name', 

In [3]:
# Roger Federer's profession
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q1426 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10833314'), ('name', 'tennis player')]
1


In [4]:
# 'tennis player' predicates
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q10833314 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direc

In [5]:
# 'tennis player' details
queryString = """
SELECT DISTINCT ?p ?name ?obj ?obj_name
WHERE {
   wd:Q10833314 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?obj_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('name', 'field of this occupation'), ('obj', 'http://www.wikidata.org/entity/Q56054951'), ('obj_name', "men's tennis")]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q18536342'), ('obj_name', 'competitive player')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28640'), ('obj_name', 'profession')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('name', 'field of this occupation'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('obj_name', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('obj_name', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q2066131'), ('obj_name', 'athlete')]
[('p', 'http://www.wikidata.org/prop/dir

Final query for this task

In [6]:
# write your final query

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q1426 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10833314'), ('name', 'tennis player')]
1


## Task 2
Identify the BGP for tennis tournament

In [7]:
# Things related to Federer
queryString = """
SELECT DISTINCT ?s ?s_name ?p ?name
WHERE {
   # bind something
   ?s ?p wd:Q1426 .
   # get the label
   ?p sc:name ?name.
   ?s sc:name ?s_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q22956224'), ('s_name', "2017 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q3983549'), ('s_name', '2007 Tennis Masters Cup – singles'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q2537671'), ('s_name', '2011 Qatar Open – singles'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q13076407'), ('s_name', "2012 Mutua Madrid Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q30311438'), ('s_name', '2017 Gerry Weber Open – singles'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q3633911'), ('s_name', '2005 Western & Southern Financial Group Masters – singles'), ('p', '

In [8]:
# '2017 Australian Open – Men's Singles' details
queryString = """
SELECT DISTINCT ?o ?o_name ?p ?name
WHERE {
   # bind something
   wd:Q22956224 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q22077628'), ('o_name', '2017 Australian Open'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('o', 'http://www.wikidata.org/entity/Q408'), ('o_name', 'Australia'), ('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('o', 'http://www.wikidata.org/entity/Q847'), ('o_name', 'tennis'), ('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('o', 'http://www.wikidata.org/entity/Q1426'), ('o_name', 'Roger Federer'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('o', 'http://www.wikidata.org/entity/Q16893072'), ('o_name', "men's singles"), ('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('o', 'http://www.wikidata.org/entity/Q46190676'), ('o_name', 'tennis event'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
6


In [9]:
# 'tennis event' details
queryString = """
SELECT DISTINCT ?o ?o_name ?p ?name
WHERE {
   # bind something
   wd:Q46190676 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q21170330'), ('o_name', 'tennis qualification event'), ('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('o', 'http://www.wikidata.org/entity/P361'), ('o_name', 'part of'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('o', 'http://www.wikidata.org/entity/Q16510064'), ('o_name', 'sporting event'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('o', 'http://www.wikidata.org/entity/Q847'), ('o_name', 'tennis'), ('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('o', 'http://www.wikidata.org/entity/P641'), ('o_name', 'sport'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('o', 'http://www.wikidata.org/entity/Q47345468'), ('o_name', 'tennis tournament edition'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('o', 'http://www.wikidata.or

In [10]:
# 'tennis tournament edition' details
queryString = """
SELECT DISTINCT ?o ?o_name ?p ?name
WHERE {
   # bind something
   wd:Q47345468 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/P393'), ('o_name', 'edition number'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('o', 'http://www.wikidata.org/entity/Q47382471'), ('o_name', 'day-by-day summary'), ('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('o', 'http://www.wikidata.org/entity/P580'), ('o_name', 'start time'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('o', 'http://www.wikidata.org/entity/P582'), ('o_name', 'end time'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('o', 'http://www.wikidata.org/entity/P17'), ('o_name', 'country'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('o', 'http://www.wikidata.org/entity/Q13219666'), ('o_name', 'tennis tournament'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('o', 'htt

In [11]:
# 'tennis tournament' details
queryString = """
SELECT DISTINCT ?o ?o_name ?p ?name
WHERE {
   # bind something
   wd:Q13219666 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q13406554'), ('o_name', 'sports competition'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('o', 'http://www.wikidata.org/entity/Q847'), ('o_name', 'tennis'), ('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('o', 'http://www.wikidata.org/entity/Q46190676'), ('o_name', 'tennis event'), ('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('o', 'http://www.wikidata.org/entity/Q6452600'), ('o_name', 'Template:ATP Tournament'), ('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('o', 'http://www.wikidata.org/entity/Q6642846'), ('o_name', 'list of tennis tournaments'), ('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('o', 'http://www.wikidata.org/entity/Q7474005'), ('o_name', 'Template:Infobox tennis tournament year'), ('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's mai

Final query for this task

In [3]:
# write your final query

queryString = """
SELECT DISTINCT ?o ?o_name
WHERE {
   # bind something
   wd:Q47345468 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/P393'), ('o_name', 'edition number')]
[('o', 'http://www.wikidata.org/entity/Q47382471'), ('o_name', 'day-by-day summary')]
[('o', 'http://www.wikidata.org/entity/P580'), ('o_name', 'start time')]
[('o', 'http://www.wikidata.org/entity/P582'), ('o_name', 'end time')]
[('o', 'http://www.wikidata.org/entity/P17'), ('o_name', 'country')]
[('o', 'http://www.wikidata.org/entity/Q13219666'), ('o_name', 'tennis tournament')]
[('o', 'http://www.wikidata.org/entity/Q27020041'), ('o_name', 'sports season')]
[('o', 'http://www.wikidata.org/entity/Q847'), ('o_name', 'tennis')]
[('o', 'http://www.wikidata.org/entity/P527'), ('o_name', 'has part')]
[('o', 'http://www.wikidata.org/entity/Q47358534'), ('o_name', 'tennis tour edition')]
[('o', 'http://www.wikidata.org/entity/P641'), ('o_name', 'sport')]
[('o', 'http://www.wikidata.org/entity/Q46190676'), ('o_name', 'tennis event')]
[('o', 'http://www.wikidata.org/entity/P276'), ('o_name', 'location')]
[('o

## Task 3
Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

In [12]:
# Things related to 'Tennis tournaments'
queryString = """
SELECT DISTINCT ?s ?s_name ?p ?name
WHERE {
   # bind something
   ?s ?p wd:Q5325422 .
   # get the label
   ?p sc:name ?name.
   ?s sc:name ?s_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q13219666'), ('s_name', 'tennis tournament'), ('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
1


In [14]:
# Things related to 'Tennis tournament'
queryString = """
SELECT DISTINCT ?s ?s_name ?p ?name
WHERE {
   # bind something
   ?s ?p wd:Q13219666 .
   # get the label
   ?p sc:name ?name.
   ?s sc:name ?s_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q299931'), ('s_name', 'Chile Open'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q299384'), ('s_name', 'Delray Beach International Tennis Championships'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q1961762'), ('s_name', 'Mersin Cup'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q2415348'), ('s_name', 'Masters France'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q3935780'), ('s_name', 'Rijeka Open'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q3954502'), ('s_name', 'Seguros Bolívar Open Cali'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('s', 'http://www.w

In [15]:
# Things related to 'Tennis tournament' having 'Wimbledon' in the subject name
queryString = """
SELECT DISTINCT ?s ?s_name ?p ?name
WHERE {
   # bind something
   ?s ?p wd:Q13219666 .
   # get the label
   ?p sc:name ?name.
   ?s sc:name ?s_name.
   FILTER regex(?s_name, "Wimbledon", "i")
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q41520'), ('s_name', 'Wimbledon Championships'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q12776'), ('s_name', 'Wimbledon Plate'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
2


In [16]:
# 'Wimbledon Championships' details
queryString = """
SELECT DISTINCT ?p ?name ?o ?o_name
WHERE {
   # bind something
   wd:Q41520 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('name', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q815369'), ('o_name', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location'), ('o', 'http://www.wikidata.org/entity/Q815369'), ('o_name', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('name', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q223041'), ('o_name', 'International Tennis Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('o_name', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('o_name', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('o_name', 'recurring spor

Final query for this task

In [17]:
# write your final query
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P361 wd:Q102113 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q43605'), ('name', 'French Open')]
[('s', 'http://www.wikidata.org/entity/Q123577'), ('name', 'US Open')]
[('s', 'http://www.wikidata.org/entity/Q41520'), ('name', 'Wimbledon Championships')]
[('s', 'http://www.wikidata.org/entity/Q60874'), ('name', 'Australian Open')]
4


## Task 4
Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

In [18]:
# "surface played on" predicate objects
queryString = """
SELECT DISTINCT ?o ?o_name
WHERE {
   # bind something
   ?s wdt:P765 ?o .
   # get the label
   ?o sc:name ?o_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q47677214'), ('o_name', 'carpet')]
[('o', 'http://www.wikidata.org/entity/Q133833'), ('o_name', 'gravel')]
[('o', 'http://www.wikidata.org/entity/Q13731184'), ('o_name', 'artificial ice rink')]
[('o', 'http://www.wikidata.org/entity/Q2306698'), ('o_name', 'Desso GrassMaster')]
[('o', 'http://www.wikidata.org/entity/Q3179760'), ('o_name', 'tarmac')]
[('o', 'http://www.wikidata.org/entity/Q190903'), ('o_name', 'herbaceous plant')]
[('o', 'http://www.wikidata.org/entity/Q198718'), ('o_name', 'clay court')]
[('o', 'http://www.wikidata.org/entity/Q838828'), ('o_name', 'parquetry')]
[('o', 'http://www.wikidata.org/entity/Q30198'), ('o_name', 'marsh')]
[('o', 'http://www.wikidata.org/entity/Q30688663'), ('o_name', 'Mondotrack')]
[('o', 'http://www.wikidata.org/entity/Q29167278'), ('o_name', 'carpet court')]
[('o', 'http://www.wikidata.org/entity/Q47676928'), ('o_name', 'clay')]
[('o', 'http://www.wikidata.org/entity/Q189259'), ('o_name', 'asphalt

In [19]:
# 'grass cout' details
queryString = """
SELECT DISTINCT ?p ?p_name ?o ?o_name
WHERE {
   # bind something
   wd:Q1151291 ?p ?o .
   # get the label
   ?o sc:name ?o_name.
   ?p sc:name ?p_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('p_name', 'uses'), ('o', 'http://www.wikidata.org/entity/Q47678177'), ('o_name', 'grass')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('p_name', 'uses'), ('o', 'http://www.wikidata.org/entity/Q47769255'), ('o_name', 'outdoor grass')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q47677460'), ('o_name', 'tennis playing surface')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('p_name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('o_name', 'tennis')]
4


In [20]:
# subclasses of 'tennis playing surface'

queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P279 wd:Q47677460 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q47677214'), ('name', 'carpet')]
[('s', 'http://www.wikidata.org/entity/Q47676928'), ('name', 'clay')]
[('s', 'http://www.wikidata.org/entity/Q1151291'), ('name', 'grass court')]
[('s', 'http://www.wikidata.org/entity/Q47678177'), ('name', 'grass')]
[('s', 'http://www.wikidata.org/entity/Q47769232'), ('name', 'indoor hard')]
[('s', 'http://www.wikidata.org/entity/Q47769271'), ('name', 'outdoor hard')]
[('s', 'http://www.wikidata.org/entity/Q47777555'), ('name', 'hardcourt surface')]
[('s', 'http://www.wikidata.org/entity/Q47769216'), ('name', 'indoor carpet')]
[('s', 'http://www.wikidata.org/entity/Q47769202'), ('name', 'indoor')]
[('s', 'http://www.wikidata.org/entity/Q47769177'), ('name', 'asphalt')]
[('s', 'http://www.wikidata.org/entity/Q47769149'), ('name', 'acrylic')]
[('s', 'http://www.wikidata.org/entity/Q47769166'), ('name', 'artificial grass')]
[('s', 'http://www.wikidata.org/entity/Q47769242'), ('name', 'outdoor')]
[('s', 'http:

In [21]:
# 'outdoor grass' details

queryString = """
SELECT DISTINCT ?p ?p_name ?o ?name
WHERE {
   # bind something
   wd:Q47769255 ?p ?o .
   # get the label
   ?o sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q47677460'), ('name', 'tennis playing surface')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q47769242'), ('name', 'outdoor')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('p_name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('name', 'tennis')]
3


In [22]:
# outdoor tennis playing surfaces

queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P279 wd:Q47677460 .
   ?s wdt:P279 wd:Q47769242 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q47769271'), ('name', 'outdoor hard')]
[('s', 'http://www.wikidata.org/entity/Q47769249'), ('name', 'outdoor clay')]
[('s', 'http://www.wikidata.org/entity/Q47769255'), ('name', 'outdoor grass')]
3


In [4]:
# "surface played on" where the sport is tennis
queryString = """
SELECT DISTINCT ?o ?o_name
WHERE {
   # bind something
   ?s wdt:P765 ?o .
   ?o wdt:P641 wd:Q847 .
   # get the label
   ?o sc:name ?o_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q47677214'), ('o_name', 'carpet')]
[('o', 'http://www.wikidata.org/entity/Q198718'), ('o_name', 'clay court')]
[('o', 'http://www.wikidata.org/entity/Q29167278'), ('o_name', 'carpet court')]
[('o', 'http://www.wikidata.org/entity/Q47676928'), ('o_name', 'clay')]
[('o', 'http://www.wikidata.org/entity/Q1330871'), ('o_name', 'hardcourt')]
[('o', 'http://www.wikidata.org/entity/Q1151291'), ('o_name', 'grass court')]
[('o', 'http://www.wikidata.org/entity/Q47678177'), ('o_name', 'grass')]
[('o', 'http://www.wikidata.org/entity/Q47769232'), ('o_name', 'indoor hard')]
[('o', 'http://www.wikidata.org/entity/Q47769271'), ('o_name', 'outdoor hard')]
[('o', 'http://www.wikidata.org/entity/Q47777555'), ('o_name', 'hardcourt surface')]
[('o', 'http://www.wikidata.org/entity/Q47769216'), ('o_name', 'indoor carpet')]
[('o', 'http://www.wikidata.org/entity/Q47769149'), ('o_name', 'acrylic')]
[('o', 'http://www.wikidata.org/entity/Q47769166'), ('o_name', 

In [6]:
# "surface played on" where the sport is tennis
queryString = """
SELECT DISTINCT ?a ?name
WHERE {
   # bind something
   ?s wdt:P765 ?o .
   ?o wdt:P641 wd:Q847 .
   ?o wdt:P2283 ?a .
   # get the label
   ?a sc:name ?name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q47677214'), ('name', 'carpet')]
[('a', 'http://www.wikidata.org/entity/Q47676928'), ('name', 'clay')]
[('a', 'http://www.wikidata.org/entity/Q47769232'), ('name', 'indoor hard')]
[('a', 'http://www.wikidata.org/entity/Q47769271'), ('name', 'outdoor hard')]
[('a', 'http://www.wikidata.org/entity/Q47777555'), ('name', 'hardcourt surface')]
[('a', 'http://www.wikidata.org/entity/Q47678177'), ('name', 'grass')]
[('a', 'http://www.wikidata.org/entity/Q47769255'), ('name', 'outdoor grass')]
7


In [11]:
# "grass" details
queryString = """
SELECT DISTINCT ?p ?name ?o ?o_name
WHERE {
   # bind something
   wd:Q47678177 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('o_name', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1535'), ('name', 'used by'), ('o', 'http://www.wikidata.org/entity/Q1151291'), ('o_name', 'grass court')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q47677460'), ('o_name', 'tennis playing surface')]
3


In [12]:
# "surface played on" where the sport is tennis
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
    ?s wdt:P641 wd:Q847 .
    ?s wdt:P1535 ?o .
   # get the label
   ?s sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q153362'), ('name', 'tennis racquet')]
[('s', 'http://www.wikidata.org/entity/Q47677214'), ('name', 'carpet')]
[('s', 'http://www.wikidata.org/entity/Q47676928'), ('name', 'clay')]
[('s', 'http://www.wikidata.org/entity/Q47678177'), ('name', 'grass')]
[('s', 'http://www.wikidata.org/entity/Q741118'), ('name', 'tennis court')]
[('s', 'http://www.wikidata.org/entity/Q47777555'), ('name', 'hardcourt surface')]
[('s', 'http://www.wikidata.org/entity/Q246514'), ('name', 'tennis ball')]
[('s', 'http://www.wikidata.org/entity/Q1246411'), ('name', 'tennis string')]
[('s', 'http://www.wikidata.org/entity/Q47755582'), ('name', 'net post')]
9


In [14]:
# "surface played on" where the sport is tennis
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
    ?s wdt:P641 wd:Q847 .
    ?s wdt:P1535 ?o .
    ?s wdt:P279 wd:Q47677460 .
   # get the label
   ?s sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q47676928'), ('name', 'clay')]
[('s', 'http://www.wikidata.org/entity/Q47677214'), ('name', 'carpet')]
[('s', 'http://www.wikidata.org/entity/Q47777555'), ('name', 'hardcourt surface')]
[('s', 'http://www.wikidata.org/entity/Q47678177'), ('name', 'grass')]
4


In [15]:

queryString = """
SELECT DISTINCT ?p ?name ?o ?o_name
WHERE {
   # bind something
    wd:Q741118 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1535'), ('name', 'used by'), ('o', 'http://www.wikidata.org/entity/Q847'), ('o_name', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('o_name', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q47677460'), ('o_name', 'tennis playing surface')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q2310214'), ('o_name', 'pitch')]
[('p', 'http://www.wikidata.org/prop/direct/P1535'), ('name', 'used by'), ('o', 'http://www.wikidata.org/entity/Q1153025'), ('o_name', 'wheelchair tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q48291351'), ('o_name', 'facet of tennis playing area')]
[('p', 'http://www.wikidata.org/prop/direct/

In [16]:

queryString = """
SELECT DISTINCT ?p ?name ?o ?o_name
WHERE {
   # bind something
    wd:Q48308960 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('o_name', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q741118'), ('o_name', 'tennis court')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q47677460'), ('o_name', 'tennis playing surface')]
3


Final query for this task

In [23]:
# write your final query
# NOTE: I restricted to the outdoor surfaces to get exactly 3 results, in the previous queries i found 4 main surfaces 
# and others to different level of detail
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P279 wd:Q47677460 .
   ?s wdt:P279 wd:Q47769242 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q47769271'), ('name', 'outdoor hard')]
[('s', 'http://www.wikidata.org/entity/Q47769249'), ('name', 'outdoor clay')]
[('s', 'http://www.wikidata.org/entity/Q47769255'), ('name', 'outdoor grass')]
3


## Task 5
Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

In [24]:
# write your queries

queryString = """
SELECT DISTINCT ?p ?name ?o ?o_name
WHERE {
   # bind something
   wd:Q41520 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('name', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q815369'), ('o_name', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location'), ('o', 'http://www.wikidata.org/entity/Q815369'), ('o_name', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('name', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q223041'), ('o_name', 'International Tennis Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('o_name', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('o_name', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('o_name', 'recurring spor

In [25]:
# Things related to 'Wimbledon Championships'

queryString = """
SELECT DISTINCT ?p ?name ?s ?s_name
WHERE {
   # bind something
   ?s ?p wd:Q41520 .
   # get the label
   ?p sc:name ?name.
   ?s sc:name ?s_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('s', 'http://www.wikidata.org/entity/Q268871'), ('s_name', '2004 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('s', 'http://www.wikidata.org/entity/Q173961'), ('s_name', '1978 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('s', 'http://www.wikidata.org/entity/Q925026'), ('s_name', '1971 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('s', 'http://www.wikidata.org/entity/Q15854765'), ('s_name', '2014 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('s', 'http://www.wikidata.org/entity/Q30085309'), ('s_name', '2018 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in'), ('s', 'http://www.wikidata.org/entity/Q437350'), ('s_name', 'D

In [26]:
# Things related to '2018 Wimbledon Championships'

queryString = """
SELECT DISTINCT ?p ?name ?s ?s_name
WHERE {
   # bind something
   ?s ?p wd:Q30085309 .
   # get the label
   ?p sc:name ?name.
   ?s sc:name ?s_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('s', 'http://www.wikidata.org/entity/Q30098268'), ('s_name', "2018 Wimbledon Championships – Women's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by'), ('s', 'http://www.wikidata.org/entity/Q25344121'), ('s_name', '2017 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('s', 'http://www.wikidata.org/entity/Q32358401'), ('s_name', '2018 Wimbledon Championships – Mixed Doubles')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('s', 'http://www.wikidata.org/entity/Q55639017'), ('s_name', '2018 Wimbledon Championships – Wheelchair Quad Doubles')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('s', 'http://www.wikidata.org/entity/Q55455418'), ('s_name', "2018 Wimbledon Championships – Wheelchair Men's Doubles")]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 

In [27]:
# '2018 Wimbledon Championships – Men's Singles' details

queryString = """
SELECT DISTINCT ?p ?name ?o ?o_name
WHERE {
   # bind something
   wd:Q54961494 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('o', 'http://www.wikidata.org/entity/Q30085309'), ('o_name', '2018 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('o_name', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('o_name', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q16893072'), ('o_name', "men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q46190676'), ('o_name', 'tennis event')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('o', 'http://www.wikidata.org/entity/Q5812'), ('o_name', 'Novak Djokovic')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'follow

Final query for this task

In [28]:
# write your final query

queryString = """
SELECT DISTINCT ?s ?name ?w ?w_name
WHERE {
   # bind something
   ?s wdt:P361 wd:Q30085309 .
   ?s wdt:P1346 ?w .
   # get the label
   ?s sc:name ?name.
   ?w sc:name ?w_name.
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q32357790'), ('name', "2018 Wimbledon Championships – Men's Doubles"), ('w', 'http://www.wikidata.org/entity/Q53342'), ('w_name', 'Mike Bryan')]
[('s', 'http://www.wikidata.org/entity/Q54961494'), ('name', "2018 Wimbledon Championships – Men's Singles"), ('w', 'http://www.wikidata.org/entity/Q5812'), ('w_name', 'Novak Djokovic')]
[('s', 'http://www.wikidata.org/entity/Q30098268'), ('name', "2018 Wimbledon Championships – Women's Singles"), ('w', 'http://www.wikidata.org/entity/Q77178'), ('w_name', 'Angelique Kerber')]
[('s', 'http://www.wikidata.org/entity/Q32356704'), ('name', "2018 Wimbledon Championships – Women's Doubles"), ('w', 'http://www.wikidata.org/entity/Q11052500'), ('w_name', 'Barbora Krejčíková')]
[('s', 'http://www.wikidata.org/entity/Q55455423'), ('name', "2018 Wimbledon Championships – Wheelchair Women's Doubles"), ('w', 'http://www.wikidata.org/entity/Q16224602'), ('w_name', 'Yui Kamiji')]
[('s', 'http://www.wikidata.org/

## Task 6
Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

In [29]:
# Grand Slam details

queryString = """
SELECT DISTINCT ?p ?name ?o ?o_name
WHERE {
   # bind something
   wd:Q102113 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 10
"""


print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q50301185'), ('o_name', 'tennis tour with multiple editions')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q7700500'), ('o_name', 'tennis tour')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q2223649'), ('o_name', 'French cheese')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q5851433'), ('o_name', 'Template:Infobox GrandSlamTournaments')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('o_name', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q123577'), ('o_name', 'US Open')]
[('p', 'http://www.wikidata.org/

In [30]:
queryString = """
SELECT DISTINCT ?w ?name (count(?t) as ?cnt)
WHERE {
   # bind something
   ?c wdt:P361 wd:Q102113 .
   ?t wdt:P361 ?c .
   ?t wdt:P1346 ?w .
   # get the label
   ?w sc:name ?name.
}
Group by ?w ?name
order by desc(?cnt)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [31]:
queryString = """
SELECT DISTINCT ?w ?name
WHERE {
   # bind something
   ?c wdt:P361 wd:Q102113 .
   ?i wdt:P31 ?c .
   ?t wdt:P361 ?i .
   ?t wdt:P1346 ?w .
   # get the label
   ?w sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('w', 'http://www.wikidata.org/entity/Q1558013'), ('name', 'Pierre Barthès')]
[('w', 'http://www.wikidata.org/entity/Q299959'), ('name', 'Eileen Bennett Whittingstall')]
[('w', 'http://www.wikidata.org/entity/Q18918709'), ('name', 'Markéta Vondroušová')]
[('w', 'http://www.wikidata.org/entity/Q193661'), ('name', 'Stan Wawrinka')]
[('w', 'http://www.wikidata.org/entity/Q53410'), ('name', 'John Doeg')]
[('w', 'http://www.wikidata.org/entity/Q54546'), ('name', 'Nancy Richey')]
[('w', 'http://www.wikidata.org/entity/Q1260231'), ('name', 'Geoff Masters')]
[('w', 'http://www.wikidata.org/entity/Q14905846'), ('name', 'Borna Ćorić')]
[('w', 'http://www.wikidata.org/entity/Q207352'), ('name', 'Agnieszka Radwańska')]
[('w', 'http://www.wikidata.org/entity/Q20968624'), ('name', 'Viktória Kužmová')]
10


In [32]:
queryString = """
SELECT DISTINCT ?w ?name (count(?t) as ?cnt)
WHERE {
   # bind something
   ?c wdt:P361 wd:Q102113 .
   ?i wdt:P31 ?c .
   ?t wdt:P361 ?i .
   ?t wdt:P1346 ?w .
   # get the label
   ?w sc:name ?name.
}
Group by ?w ?name
order by desc(?cnt)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('w', 'http://www.wikidata.org/entity/Q54545'), ('name', 'Martina Navratilova'), ('cnt', '58')]
[('w', 'http://www.wikidata.org/entity/Q202342'), ('name', 'Margaret Court'), ('cnt', '48')]
[('w', 'http://www.wikidata.org/entity/Q11459'), ('name', 'Serena Williams'), ('cnt', '39')]
[('w', 'http://www.wikidata.org/entity/Q959958'), ('name', 'Shingo Kunieda'), ('cnt', '32')]
[('w', 'http://www.wikidata.org/entity/Q134720'), ('name', 'Martina Hingis'), ('cnt', '25')]
[('w', 'http://www.wikidata.org/entity/Q54527'), ('name', 'Billie Jean King'), ('cnt', '23')]
[('w', 'http://www.wikidata.org/entity/Q11662'), ('name', 'Steffi Graf'), ('cnt', '23')]
[('w', 'http://www.wikidata.org/entity/Q16224602'), ('name', 'Yui Kamiji'), ('cnt', '22')]
[('w', 'http://www.wikidata.org/entity/Q1426'), ('name', 'Roger Federer'), ('cnt', '22')]
[('w', 'http://www.wikidata.org/entity/Q54543'), ('name', 'Helen Wills'), ('cnt', '21')]
10


In [33]:
queryString = """
SELECT DISTINCT ?w ?name ?bd (count(?t) as ?cnt)
WHERE {
   # bind something
   ?c wdt:P361 wd:Q102113 .
   ?i wdt:P31 ?c .
   ?t wdt:P361 ?i .
   ?t wdt:P1346 ?w .
   ?w wdt:P569 ?bd .
   # get the label
   ?w sc:name ?name.
    
}
Group by ?w ?name ?bd
order by desc(?cnt)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('w', 'http://www.wikidata.org/entity/Q54545'), ('name', 'Martina Navratilova'), ('bd', '1956-10-18T00:00:00Z'), ('cnt', '58')]
[('w', 'http://www.wikidata.org/entity/Q202342'), ('name', 'Margaret Court'), ('bd', '1942-07-16T00:00:00Z'), ('cnt', '48')]
[('w', 'http://www.wikidata.org/entity/Q11459'), ('name', 'Serena Williams'), ('bd', '1981-09-26T00:00:00Z'), ('cnt', '39')]
[('w', 'http://www.wikidata.org/entity/Q959958'), ('name', 'Shingo Kunieda'), ('bd', '1984-02-21T00:00:00Z'), ('cnt', '32')]
[('w', 'http://www.wikidata.org/entity/Q134720'), ('name', 'Martina Hingis'), ('bd', '1980-09-30T00:00:00Z'), ('cnt', '25')]
[('w', 'http://www.wikidata.org/entity/Q11662'), ('name', 'Steffi Graf'), ('bd', '1969-06-14T00:00:00Z'), ('cnt', '23')]
[('w', 'http://www.wikidata.org/entity/Q54527'), ('name', 'Billie Jean King'), ('bd', '1943-11-22T00:00:00Z'), ('cnt', '23')]
[('w', 'http://www.wikidata.org/entity/Q16224602'), ('name', 'Yui Kamiji'), ('bd', '1994-04-24T00:00:00Z'), ('cnt', 

In [34]:
# Top 10 winners born after 1975
queryString = """
SELECT DISTINCT ?w ?name ?bd (count(?t) as ?cnt)
WHERE {
   # bind something
   ?c wdt:P361 wd:Q102113 .
   ?i wdt:P31 ?c .
   ?t wdt:P361 ?i .
   ?t wdt:P1346 ?w .
   ?w wdt:P569 ?bd .
   # get the label
   ?w sc:name ?name.
   FILTER (?bd >= "1975-01-01T00:00:00Z"^^xsd:dateTime)
}
Group by ?w ?name ?bd
order by desc(?cnt)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('w', 'http://www.wikidata.org/entity/Q11459'), ('name', 'Serena Williams'), ('bd', '1981-09-26T00:00:00Z'), ('cnt', '39')]
[('w', 'http://www.wikidata.org/entity/Q959958'), ('name', 'Shingo Kunieda'), ('bd', '1984-02-21T00:00:00Z'), ('cnt', '32')]
[('w', 'http://www.wikidata.org/entity/Q134720'), ('name', 'Martina Hingis'), ('bd', '1980-09-30T00:00:00Z'), ('cnt', '25')]
[('w', 'http://www.wikidata.org/entity/Q16224602'), ('name', 'Yui Kamiji'), ('bd', '1994-04-24T00:00:00Z'), ('cnt', '22')]
[('w', 'http://www.wikidata.org/entity/Q1426'), ('name', 'Roger Federer'), ('bd', '1981-08-08T00:00:00Z'), ('cnt', '22')]
[('w', 'http://www.wikidata.org/entity/Q259606'), ('name', 'Esther Vergeer'), ('bd', '1981-07-18T00:00:00Z'), ('cnt', '21')]
[('w', 'http://www.wikidata.org/entity/Q11578'), ('name', 'Venus Williams'), ('bd', '1980-06-17T00:00:00Z'), ('cnt', '20')]
[('w', 'http://www.wikidata.org/entity/Q10132'), ('name', 'Rafael Nadal'), ('bd', '1986-06-03T00:00:00Z'), ('cnt', '20')]
[

Final query for this task

In [35]:
# write your final query

queryString = """
SELECT DISTINCT ?w ?name (count(?t) as ?wins)
WHERE {
   # bind something
   ?c wdt:P361 wd:Q102113 .
   ?i wdt:P31 ?c .
   ?t wdt:P361 ?i .
   ?t wdt:P1346 ?w .
   ?w wdt:P569 ?bd .
   # get the label
   ?w sc:name ?name.
   FILTER (?bd >= "1975-01-01T00:00:00Z"^^xsd:dateTime)
}
Group by ?w ?name
order by desc(?wins)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('w', 'http://www.wikidata.org/entity/Q11459'), ('name', 'Serena Williams'), ('wins', '39')]
[('w', 'http://www.wikidata.org/entity/Q959958'), ('name', 'Shingo Kunieda'), ('wins', '32')]
[('w', 'http://www.wikidata.org/entity/Q134720'), ('name', 'Martina Hingis'), ('wins', '25')]
[('w', 'http://www.wikidata.org/entity/Q16224602'), ('name', 'Yui Kamiji'), ('wins', '22')]
[('w', 'http://www.wikidata.org/entity/Q1426'), ('name', 'Roger Federer'), ('wins', '22')]
[('w', 'http://www.wikidata.org/entity/Q259606'), ('name', 'Esther Vergeer'), ('wins', '21')]
[('w', 'http://www.wikidata.org/entity/Q5812'), ('name', 'Novak Djokovic'), ('wins', '20')]
[('w', 'http://www.wikidata.org/entity/Q10132'), ('name', 'Rafael Nadal'), ('wins', '20')]
[('w', 'http://www.wikidata.org/entity/Q11578'), ('name', 'Venus Williams'), ('wins', '20')]
[('w', 'http://www.wikidata.org/entity/Q53340'), ('name', 'Bob Bryan'), ('wins', '18')]
10


## Task 7
Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

In [36]:
# Sport property

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q5812 wdt:P641 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q847'), ('name', 'tennis')]
1


In [37]:
# Sport property details

queryString = """
SELECT DISTINCT ?s ?s_name ?o ?name
WHERE {
   # bind something
   wd:Q847 ?s ?o .
   # get the label
   ?o sc:name ?name.
   ?s sc:name ?s_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P1151'), ('s_name', "topic's main Wikimedia portal"), ('o', 'http://www.wikidata.org/entity/Q8288752'), ('name', 'Portal:Tennis')]
[('s', 'http://www.wikidata.org/prop/direct/P1343'), ('s_name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('name', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('s', 'http://www.wikidata.org/prop/direct/P1424'), ('s_name', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q8087332'), ('name', 'Template:Tennis-stub')]
[('s', 'http://www.wikidata.org/prop/direct/P1424'), ('s_name', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q10943918'), ('name', 'Template:Tennis box')]
[('s', 'http://www.wikidata.org/prop/direct/P1889'), ('s_name', 'different from'), ('o', 'http://www.wikidata.org/entity/Q12565374'), ('name', 'Tenis')]
[('s', 'http://www.wikidata.org/prop/direct/P2184'), ('s_name', 'history of topic'), ('o', 'http://www.wikidata.org/e

In [38]:
# Contry for sport property

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q5812 wdt:P1532 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q403'), ('name', 'Serbia')]
1


In [39]:
# Contries that won at least one super slam

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   ?c wdt:P361 wd:Q102113 .
   ?i wdt:P31 ?c .
   ?t wdt:P361 ?i .
   ?t wdt:P1346 ?w .
   ?w wdt:P1532 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q216923'), ('name', 'Chinese Taipei')]
[('o', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('o', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('o', 'http://www.wikidata.org/entity/Q148'), ('name', "People's Republic of China")]
[('o', 'http://www.wikidata.org/entity/Q15180'), ('name', 'Soviet Union')]
[('o', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil')]
[('o', 'http://www.wikidata.org/entity/Q159'), ('name', 'Russia')]
[('o', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada')]
[('o', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan')]
[('o', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('o', 'http://www.wikidata.org/entity/Q184'), ('name', 'Belarus')]
[('o', 'http://www.wikidata.org/entity/Q20'), ('name', 'Norway')]
[('o', 'http://www.wikidata.org/entity/Q21'), ('name', 'England')]
[('o', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('o', 'h

In [40]:
# Players and count of different Grand Slam win

queryString = """
SELECT DISTINCT ?w ?w_name (count(Distinct ?c) as ?cnt)
WHERE {
   # bind something
   ?c wdt:P361 wd:Q102113 .
   ?i wdt:P31 ?c .
   ?t wdt:P361 ?i .
   ?t wdt:P1346 ?w .
   # get the label
   ?w sc:name ?w_name.
}
group by ?w ?w_name

order by desc(?cnt)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('w', 'http://www.wikidata.org/entity/Q312728'), ('w_name', 'Ken Rosewall'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q10118'), ('w_name', 'Victoria Azarenka'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q53338'), ('w_name', 'Louise Brough'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q11578'), ('w_name', 'Venus Williams'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q1426'), ('w_name', 'Roger Federer'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q2093033'), ('w_name', 'Pierre-Hugues Herbert'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q5812'), ('w_name', 'Novak Djokovic'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q11459'), ('w_name', 'Serena Williams'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q270414'), ('w_name', 'Anne Smith'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q959958'), ('w_name', 'Shingo Kunieda'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q2326828'), ('w_n

In [41]:
# Query to check the previous result

queryString = """
SELECT DISTINCT ?w ?w_name ?c ?name
WHERE {
   # bind something
   ?c wdt:P361 wd:Q102113 .
   ?i wdt:P31 ?c .
   ?t wdt:P361 ?i .
   ?t wdt:P1346 ?w .
   # get the label
   ?c sc:name ?name.
   ?w sc:name ?w_name.
}

order by ?w
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('w', 'http://www.wikidata.org/entity/Q10118'), ('w_name', 'Victoria Azarenka'), ('c', 'http://www.wikidata.org/entity/Q43605'), ('name', 'French Open')]
[('w', 'http://www.wikidata.org/entity/Q10118'), ('w_name', 'Victoria Azarenka'), ('c', 'http://www.wikidata.org/entity/Q60874'), ('name', 'Australian Open')]
[('w', 'http://www.wikidata.org/entity/Q10118'), ('w_name', 'Victoria Azarenka'), ('c', 'http://www.wikidata.org/entity/Q123577'), ('name', 'US Open')]
[('w', 'http://www.wikidata.org/entity/Q10118'), ('w_name', 'Victoria Azarenka'), ('c', 'http://www.wikidata.org/entity/Q41520'), ('name', 'Wimbledon Championships')]
[('w', 'http://www.wikidata.org/entity/Q10120'), ('w_name', 'Laura Robson'), ('c', 'http://www.wikidata.org/entity/Q41520'), ('name', 'Wimbledon Championships')]
[('w', 'http://www.wikidata.org/entity/Q10125'), ('w_name', 'Andy Murray'), ('c', 'http://www.wikidata.org/entity/Q41520'), ('name', 'Wimbledon Championships')]
[('w', 'http://www.wikidata.org/enti

In [42]:
# Players with 4 different Grand Slam win

queryString = """
SELECT DISTINCT ?w ?w_name (count(Distinct ?c) as ?cnt)
WHERE {
   # bind something
   ?c wdt:P361 wd:Q102113 .
   ?i wdt:P31 ?c .
   ?t wdt:P361 ?i .
   ?t wdt:P1346 ?w .
   # get the label
   ?w sc:name ?w_name.
}
group by ?w ?w_name
HAVING (count(Distinct ?c)=4)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('w', 'http://www.wikidata.org/entity/Q312728'), ('w_name', 'Ken Rosewall'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q10118'), ('w_name', 'Victoria Azarenka'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q53338'), ('w_name', 'Louise Brough'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q11578'), ('w_name', 'Venus Williams'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q1426'), ('w_name', 'Roger Federer'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q2093033'), ('w_name', 'Pierre-Hugues Herbert'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q5812'), ('w_name', 'Novak Djokovic'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q11459'), ('w_name', 'Serena Williams'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q270414'), ('w_name', 'Anne Smith'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q959958'), ('w_name', 'Shingo Kunieda'), ('cnt', '4')]
[('w', 'http://www.wikidata.org/entity/Q2326828'), ('w_n

In [43]:
# Country of players with 4 different Grand Slam win

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   ?w wdt:P1532 ?o .
   # get the label
   ?o sc:name ?name .
   {
       SELECT DISTINCT ?w
        WHERE {
           # bind something
           ?c wdt:P361 wd:Q102113 .
           ?i wdt:P31 ?c .
           ?t wdt:P361 ?i .
           ?t wdt:P1346 ?w .
        }
        group by ?w ?w_name
        HAVING (count(Distinct ?c)=4)
   }
}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('o', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('o', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil')]
[('o', 'http://www.wikidata.org/entity/Q159'), ('name', 'Russia')]
[('o', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan')]
[('o', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('o', 'http://www.wikidata.org/entity/Q184'), ('name', 'Belarus')]
[('o', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('o', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('o', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('o', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary')]
[('o', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain')]
[('o', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('o', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium')]
[('o', 'http://www.

Final query for this task

In [44]:
# write your final query

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   ?w wdt:P1532 ?o .
   # get the label
   ?o sc:name ?name .
   {
       SELECT DISTINCT ?w
        WHERE {
           # bind something
           ?c wdt:P361 wd:Q102113 .
           ?i wdt:P31 ?c .
           ?t wdt:P361 ?i .
           ?t wdt:P1346 ?w .
        }
        group by ?w ?w_name
        HAVING (count(Distinct ?c)=4)
   }
}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('o', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('o', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil')]
[('o', 'http://www.wikidata.org/entity/Q159'), ('name', 'Russia')]
[('o', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan')]
[('o', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('o', 'http://www.wikidata.org/entity/Q184'), ('name', 'Belarus')]
[('o', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('o', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('o', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('o', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary')]
[('o', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain')]
[('o', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('o', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium')]
[('o', 'http://www.

## Task 8
Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

In [45]:
# Ranking data

queryString = """
SELECT DISTINCT ?s ?name ?o ?o_name
WHERE {
   # bind something
   ?s wdt:P1352 ?o .
   # get the label
   ?s sc:name ?name .
   ?o sc:name ?o_name .
}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [46]:
# Ranking data

queryString = """
SELECT DISTINCT ?s ?name ?o
WHERE {
   # bind something
   ?s wdt:P1352 ?o .
   # get the label
   ?s sc:name ?name .
}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q459119'), ('name', 'Chinese Taipei national football team'), ('o', '135')]
[('s', 'http://www.wikidata.org/entity/Q459119'), ('name', 'Chinese Taipei national football team'), ('o', '143')]
[('s', 'http://www.wikidata.org/entity/Q459119'), ('name', 'Chinese Taipei national football team'), ('o', '144')]
[('s', 'http://www.wikidata.org/entity/Q459119'), ('name', 'Chinese Taipei national football team'), ('o', '147')]
[('s', 'http://www.wikidata.org/entity/Q459119'), ('name', 'Chinese Taipei national football team'), ('o', '148')]
[('s', 'http://www.wikidata.org/entity/Q459119'), ('name', 'Chinese Taipei national football team'), ('o', '149')]
[('s', 'http://www.wikidata.org/entity/Q459119'), ('name', 'Chinese Taipei national football team'), ('o', '150')]
[('s', 'http://www.wikidata.org/entity/Q459119'), ('name', 'Chinese Taipei national football team'), ('o', '151')]
[('s', 'http://www.wikidata.org/entity/Q459119'), ('name', 'Chinese Taip

In [47]:
# Roger Federer ranking data

queryString = """
SELECT DISTINCT ?o
WHERE {
   # bind something
   wd:Q1426 wdt:P1352 ?o .
}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('o', '1')]
[('o', '24')]
2


In [48]:
# Tennis players having ranking 1

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?p wdt:P106 wd:Q10833314 .
   ?p wdt:P1352 1 .
   ?p sc:name ?name .
}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q207352'), ('name', 'Agnieszka Radwańska')]
[('p', 'http://www.wikidata.org/entity/Q455136'), ('name', 'Anders Järryd')]
[('p', 'http://www.wikidata.org/entity/Q455411'), ('name', 'Tom Okker')]
[('p', 'http://www.wikidata.org/entity/Q310419'), ('name', 'Nenad Zimonjić')]
[('p', 'http://www.wikidata.org/entity/Q53393'), ('name', 'Jimmy Connors')]
[('p', 'http://www.wikidata.org/entity/Q10118'), ('name', 'Victoria Azarenka')]
[('p', 'http://www.wikidata.org/entity/Q13538'), ('name', 'Henri Kontinen')]
[('p', 'http://www.wikidata.org/entity/Q1743002'), ('name', 'Robert Seguso')]
[('p', 'http://www.wikidata.org/entity/Q189542'), ('name', 'Stefan Edberg')]
[('p', 'http://www.wikidata.org/entity/Q232566'), ('name', 'Hsieh Su-wei')]
[('p', 'http://www.wikidata.org/entity/Q289365'), ('name', 'Karolína Plíšková')]
[('p', 'http://www.wikidata.org/entity/Q30217670'), ('name', 'Whitney Osuigwe')]
[('p', 'http://www.wikidata.org/entity/Q11662'), ('name

In [49]:
# Countries of tennis players having ranking 1

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   ?p wdt:P106 wd:Q10833314 .
   ?p wdt:P1352 1 .
   ?p wdt:P1532 ?o .
   # get the label
   ?o sc:name ?name .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q216923'), ('name', 'Chinese Taipei')]
[('o', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('o', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('o', 'http://www.wikidata.org/entity/Q148'), ('name', "People's Republic of China")]
[('o', 'http://www.wikidata.org/entity/Q15180'), ('name', 'Soviet Union')]
[('o', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil')]
[('o', 'http://www.wikidata.org/entity/Q159'), ('name', 'Russia')]
[('o', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada')]
[('o', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan')]
[('o', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('o', 'http://www.wikidata.org/entity/Q184'), ('name', 'Belarus')]
[('o', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('o', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('o', 'http://www.wikidata.org/entity/Q212'), ('name', 'Ukraine')]
[('o',

Final query for this task

In [50]:
# write your final query

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   ?p wdt:P106 wd:Q10833314 .
   ?p wdt:P1352 1 .
   ?p wdt:P1532 ?o .
   # get the label
   ?o sc:name ?name .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q216923'), ('name', 'Chinese Taipei')]
[('o', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('o', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('o', 'http://www.wikidata.org/entity/Q148'), ('name', "People's Republic of China")]
[('o', 'http://www.wikidata.org/entity/Q15180'), ('name', 'Soviet Union')]
[('o', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil')]
[('o', 'http://www.wikidata.org/entity/Q159'), ('name', 'Russia')]
[('o', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada')]
[('o', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan')]
[('o', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('o', 'http://www.wikidata.org/entity/Q184'), ('name', 'Belarus')]
[('o', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('o', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('o', 'http://www.wikidata.org/entity/Q212'), ('name', 'Ukraine')]
[('o',